In [25]:
import os
import cv2
import numpy as np
from typing import Generator
import pandas as pd

In [26]:
TRANSFORMED_DATA_103_PATH = "C:/Users/diogo/Desktop/Tese/Resultados/transformed_coords_103.csv"
VIDEOS_103_DIR_PATH = "C:/Users/diogo/Desktop/Tese/Dados/Videos/14.03.2022/20220314_1.03_blurred"
SOURCE_VIDEO_PATH = "C:/Users/diogo/Desktop/Tese/Dados/Videos/14.03.2022/20220314_1.03_blurred/20220314_1.03_1_11_blurred.mp4"

#### Video Utils

In [27]:
def get_video_capture(video_path: str) -> cv2.VideoCapture:
    return cv2.VideoCapture(video_path)

def generate_frames(video_capture: cv2.VideoCapture) -> Generator[np.ndarray, None, None]:
    while video_capture.isOpened():
        success, frame = video_capture.read()
        if not success:
            break
        yield cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

In [28]:
def load_and_sort_df(path):
    df = pd.read_csv(path)
    pd.to_datetime(df["time"], format="%H:%M:%S")
    return df.sort_values(["Customer_id", "new_frame_id"], ascending=True, ignore_index=True)
    

def get_video_numbers_sorted():
    videos = os.listdir(VIDEOS_103_DIR_PATH)
    video_numbers = []
    for video_name in videos:
        result = video_name[14:].split("_")
        video_numbers.append((int(result[0]), int(result[1])))
    return sorted(video_numbers)

In [29]:
def analyze_framelist(video_name, frame_list, df_list, row_id):
    frame_count = 0
    for frame in frame_list:
        if frame_count == df_list[row_id][26]: # to_analyze[0][26] corresponde ao new_frame_id
            start_point = (int(df_list[row_id][9]), int(df_list[row_id][10]))
            end_point = (int(df_list[row_id][11]), int(df_list[row_id][12]))
            color = (255, 0, 0)
            image_drawn = cv2.rectangle(frame, start_point, end_point, color, thickness=2)
            cv2.imshow("image drawn", image_drawn)
            pressed_key = cv2.waitKey(0)
            cv2.destroyAllWindows()
            if(pressed_key == 121): #pressed_key == "Y"
                customer_id = df_list[row_id][18]
                while df_list[row_id][18] == customer_id:
                    df_list[row_id].append(video_name)
                    print("row adicionada = " + str(df_list[row_id]))
                    row_id += 1
            else: 
                break # passa para o prox video 

        frame_count += 1
    return df_list, row_id


def anotate_coords(df):
    df_list = df.values.tolist()
    row_id = 0
    for numbers in get_video_numbers_sorted():   #itera pelos videos
        video_name = f"20220314_1.03_{numbers[0]}_{numbers[1]}_blurred.mp4"
        video_capture = get_video_capture(f"{VIDEOS_103_DIR_PATH}/{video_name}")
        frame_list = list(generate_frames(video_capture))
        df_list, row_id = analyze_framelist(video_name, frame_list, df_list, row_id)

In [30]:
df = load_and_sort_df(TRANSFORMED_DATA_103_PATH)
anotate_coords(df)

row adicionada = [162, 0, 194.33029174804688, 515.5386352539062, 351.9032592773437, 753.2744750976562, 0.6577473878860474, 0.0, 'person', 194.0, 515.0, 351.0, 753.0, 194.33029174804688, 515.5386352539062, 351.9032592773437, 753.2744750976562, 2219, 3.0, '10:03:42', 273.0, 753.0, 1096.3773990890709, 1359.9875918019309, 1.03, 1.0, 544, '20220314_1.03_1_6_blurred.mp4']
row adicionada = [163, 0, 188.06021118164065, 484.3260803222656, 353.35760498046875, 732.1746826171875, 0.7437835335731506, 0.0, 'person', 189.0, 488.0, 352.0, 734.0, 188.06021118164065, 484.3260803222656, 353.35760498046875, 732.1746826171875, 2223, 3.0, '10:03:43', 270.0, 732.0, 1097.1197085118663, 1364.4382066784267, 1.03, 1.0, 545, '20220314_1.03_1_6_blurred.mp4']
row adicionada = [164, 0, 191.9320373535156, 487.4489135742188, 347.1077880859375, 713.021484375, 0.4426746964454651, 0.0, 'person', 193.0, 486.0, 346.0, 717.0, 191.9320373535156, 487.4489135742188, 347.1077880859375, 713.021484375, 2227, 3.0, '10:03:44', 269.

KeyboardInterrupt: 